<a href="https://colab.research.google.com/github/tmyok/Colaboratory/blob/master/pytorch_onnx_export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install onnxruntime

     |████████████████████████████████| 3.7MB 2.8MB/s 
     |████████████████████████████████| 7.4MB 36.8MB/s 


In [2]:
import numpy as np
import torch
import onnxruntime

print("numpy: ", np.__version__)
print("torch: ", torch.__version__)
print("onnxruntime: ", onnxruntime.__version__)

numpy:  1.18.4
torch:  1.5.0+cu101
onnxruntime:  1.2.0


Verify that ONNX Runtime and PyTorch are computing the same value for the network.

In [0]:
def verify_onnx_export(pytorch_model):

  def to_numpy(tensor):
      return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()
  
  # Input to the model
  torch_in = torch.randn(1, 3, 256, 256)

  # compute PyTorch output prediction
  pytorch_model.eval()
  torch_out = pytorch_model(torch_in)

  # Export the model
  torch.onnx.export(pytorch_model, torch_in, "sample.onnx", opset_version=11)
  
  # compute ONNX Runtime output prediction
  ort_session = onnxruntime.InferenceSession("sample.onnx")
  ort_outs = ort_session.run(None, {ort_session.get_inputs()[0].name: to_numpy(torch_in)})

  # compare ONNX Runtime and PyTorch results
  np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

  print("Exported model has been tested with ONNXRuntime, and the result looks good!")

First step: simple CNN

In [4]:
class ModelC(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ModelC, self).__init__()
        self.conv = torch.nn.Conv2d(in_channels=in_channels, out_channels=out_channels,  kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        y = self.conv(x)
        return y

verify_onnx_export(ModelC(in_channels=3, out_channels=64))

Exported model has been tested with ONNXRuntime, and the result looks good!


Conv + Bn + Relu

In [5]:
class ModelCBR(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ModelCBR, self).__init__()

        self.conv = torch.nn.Conv2d(in_channels=in_channels, out_channels=out_channels,  kernel_size=3, stride=1, padding=1)
        self.bn = torch.nn.BatchNorm2d(out_channels)

    def forward(self, x):
        y = torch.nn.functional.relu(self.bn(self.conv(x)))
        return y

verify_onnx_export(ModelCBR(in_channels=3, out_channels=64))

Exported model has been tested with ONNXRuntime, and the result looks good!


CBR*2 + Pool

In [6]:
class ModelCBR2P(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ModelCBR2P, self).__init__()

        self.layer1 = ModelCBR(in_channels=in_channels, out_channels=out_channels)
        self.layer2 = ModelCBR(in_channels=out_channels, out_channels=out_channels)
        self.pool = torch.nn.MaxPool2d(2)

    def forward(self, x):
        y = self.pool(self.layer2(self.layer1(x)))
        return y

verify_onnx_export(ModelCBR2P(in_channels=3, out_channels=64))

Exported model has been tested with ONNXRuntime, and the result looks good!


Simple encoder

In [7]:
class EncoderD4(torch.nn.Module):
    def __init__(self, in_channels, base_channel_size):
        super(EncoderD4, self).__init__()

        module1_channel_size = base_channel_size
        module2_channel_size = base_channel_size * 2
        module3_channel_size = base_channel_size * 4
        module4_channel_size = base_channel_size * 8
        self.module1 = ModelCBR2P(in_channels=in_channels, out_channels=module1_channel_size)
        self.module2 = ModelCBR2P(in_channels=module1_channel_size, out_channels=module2_channel_size)
        self.module3 = ModelCBR2P(in_channels=module2_channel_size, out_channels=module3_channel_size)
        self.module4 = ModelCBR2P(in_channels=module3_channel_size, out_channels=module4_channel_size)

    def forward(self, x):
        h1 = self.module1(x)
        h2 = self.module2(h1)
        h3 = self.module3(h2)
        y = self.module4(h3)
        return y

verify_onnx_export(EncoderD4(in_channels=3, base_channel_size=64))

Exported model has been tested with ONNXRuntime, and the result looks good!


CBR*2 + Upsample

In [8]:
class ModelCBR2UP(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ModelCBR2UP, self).__init__()

        self.layer1 = ModelCBR(in_channels=in_channels, out_channels=out_channels)
        self.layer2 = ModelCBR(in_channels=out_channels, out_channels=out_channels)
        self.up = torch.nn.UpsamplingNearest2d(scale_factor=2)

    def forward(self, x):
        y = self.up(self.layer2(self.layer1(x)))
        return y

verify_onnx_export(ModelCBR2UP(in_channels=3, out_channels=64))

Exported model has been tested with ONNXRuntime, and the result looks good!


Simple decoder

In [0]:
class DecoderU4(torch.nn.Module):
    def __init__(self, in_channels, base_channel_size):
        super(DecoderU4, self).__init__()

        module4_channel_size = base_channel_size * 8
        module3_channel_size = base_channel_size * 4
        module2_channel_size = base_channel_size * 2
        module1_channel_size = base_channel_size
        self.module4 = ModelCBR2UP(in_channels=module4_channel_size, out_channels=module3_channel_size)
        self.module3 = ModelCBR2UP(in_channels=module3_channel_size, out_channels=module2_channel_size)
        self.module2 = ModelCBR2UP(in_channels=module2_channel_size, out_channels=module1_channel_size)
        self.module1 = ModelCBR2UP(in_channels=module1_channel_size, out_channels=in_channels)

    def forward(self, x):
        h1 = self.module4(x)
        h2 = self.module3(h1)
        h3 = self.module2(h2)
        y = self.module1(h3)
        return y

Autoencoder

In [10]:
class AutoEncoder(torch.nn.Module):
    def __init__(self, in_channels, base_channel_size):
        super(AutoEncoder, self).__init__()

        self.Encoder = EncoderD4(in_channels=3, base_channel_size=64)
        self.Decoder = DecoderU4(in_channels=3, base_channel_size=64)

    def forward(self, x):
        y = self.Decoder(self.Encoder(x))
        return y

verify_onnx_export(AutoEncoder(in_channels=3, base_channel_size=64))

Exported model has been tested with ONNXRuntime, and the result looks good!


Variational autoencoder

In [11]:
class VAE(torch.nn.Module):
    def __init__(self, in_channels, base_channel_size):
        super(VAE, self).__init__()

        self.Encoder = EncoderD4(in_channels=3, base_channel_size=64)
        self.Decoder = DecoderU4(in_channels=3, base_channel_size=64)

        # latent
        feature_size = nz = base_channel_size * 8
        self.mu_cnv = torch.nn.Conv2d(in_channels=feature_size, out_channels=nz, kernel_size=1, stride=1, padding=0)
        self.logvar_cnv = torch.nn.Conv2d(in_channels=feature_size, out_channels=nz, kernel_size=1, stride=1, padding=0)

    def latent(self, x):
        return self.mu_cnv(x), self.logvar_cnv(x)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu# + eps*std

    def forward(self, x):
        mu, logvar = self.latent(self.Encoder(x))
        z = self.reparameterize(mu, logvar)
        y = self.Decoder(z)
        return y

verify_onnx_export(VAE(in_channels=3, base_channel_size=64))

Exported model has been tested with ONNXRuntime, and the result looks good!
